# ETL Bronze to Silver
## 1. Importações

In [1]:
import pandas as pd
import numpy as np 
import psycopg2
from psycopg2.extras import execute_batch
import os 
import unicodedata

## 2. Configuração do Banco de Dados


In [2]:
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'social_media',
    'user': 'postgres',
    'password': 'postgres'
}

INPUT_FILE = '../Data layer/Raw/dados_brutos.csv'
OUTPUT_DIR = 'Data layer/silver/'
OUTPUT_FILE_CSV = 'data_silver.csv'

## 3. Extração de dados 

**O processo ETL foi iniciado carregando os dados CSV da pasta Raw com dados brutos**
Foi verificado o tamanho do dataset para enteder o volume de dados que estão sendo tratados 

In [3]:
print("="*80)
print("ETL BRONZE -> SILVER")
print("="*80)

print("\nExtraindo dados Bronze...")
dfR = pd.read_csv(INPUT_FILE)
print(f"   Extraido: {dfR.shape[0]:,} linhas x {dfR.shape[1]} colunas")
print(f"   Memória: {dfR.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

rows_initial = len(dfR)      

ETL BRONZE -> SILVER

Extraindo dados Bronze...
   Extraido: 1,547,896 linhas x 58 colunas
   Memória: 2101.60 MB


##  4. Transformação dos dados
Tratamento dos dados para a melhor utilização de acordo com o nosso foco.


## 4.1 Remoção de colunas que não seriam interessantes para nosso foco
Como nossa regra de negocio é focada em questões de saúde algumas colunas como quantidade de seguidores, se o usuario tem ou não filhos ou se ele tem uma conta premiun não interessa ao estudo e apenas tornam nossa base de dados mais pesada, por isso elas seram removidas.



In [4]:
df_limpo = dfR.drop(columns=['app_name','education_level','has_children','travel_frequency_per_year','posts_created_per_week','dms_sent_per_week','dms_received_per_week','time_on_messages_per_day','notification_response_rate','account_creation_year','privacy_setting_level','two_factor_auth_enabled','biometric_login_used','subscription_status','linked_accounts_count','average_session_length_minutes','last_login_date','hobbies_count','uses_premium_features', 'sessions_per_day',
 'likes_given_per_day', 'comments_written_per_day',])
print(f"A nova tabela tem : {df_limpo.shape[0]:,} linhas x {df_limpo.shape[1]} colunas")
print(df_limpo.columns)

A nova tabela tem : 1,547,896 linhas x 36 colunas
Index(['user_id', 'age', 'gender', 'country', 'urban_rural', 'income_level',
       'employment_status', 'relationship_status', 'exercise_hours_per_week',
       'sleep_hours_per_night', 'diet_quality', 'smoking', 'alcohol_frequency',
       'perceived_stress_score', 'self_reported_happiness', 'body_mass_index',
       'blood_pressure_systolic', 'blood_pressure_diastolic',
       'daily_steps_count', 'weekly_work_hours', 'social_events_per_month',
       'books_read_per_year', 'volunteer_hours_per_month',
       'daily_active_minutes_instagram', 'reels_watched_per_day',
       'stories_viewed_per_day', 'ads_viewed_per_day', 'ads_clicked_per_day',
       'time_on_feed_per_day', 'time_on_explore_per_day',
       'time_on_reels_per_day', 'followers_count', 'following_count',
       'content_type_preference', 'preferred_content_theme',
       'user_engagement_score'],
      dtype='object')


Agora temos uma tabela mais limpa e com colunas que contribuem de alguma forma para o objetivo.

## 4.2 Especificação maior do países estudados

Decidimos que nossa regra negocio se concentraria em países das americas (Brasil,Canadá e Estados Unidos)

In [5]:
df_limpo = df_limpo[(df_limpo['country'] == 'Brazil') | (df_limpo['country'] == 'Canada') | (df_limpo['country'] == 'United States')]
print(f"Extraido: {df_limpo.shape[0]:,} linhas x {df_limpo.shape[1]} colunas")
print(df_limpo['country'].unique())

Extraido: 681,671 linhas x 36 colunas
['Canada' 'United States' 'Brazil']


## 4.3 Remoção de menores de idade
Para nossa regra de negócio não seria interessante esses usuários.

In [6]:
df_limpo = df_limpo[df_limpo['age'] >= 18]
print(f"Extraido: {df_limpo.shape[0]:,} linhas x {df_limpo.shape[1]} colunas")
print(f"   Memoria: {df_limpo.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Extraido: 616,883 linhas x 36 colunas
   Memoria: 484.73 MB


## 4.4 Remoção de outliers

Foram detectados outliers em algumas colunas importantes para a construção de nosso dashboard por isso precisamos removê-los 

In [7]:
for col in df_limpo.select_dtypes(include=["float64","int64"]).columns:
    Q1 = df_limpo[col].quantile(0.25)
    Q3 = df_limpo[col].quantile(0.75)
    IQR = Q3 - Q1

    limite_inf = Q1 - 1.5 * IQR
    limite_sup = Q3 + 1.5 * IQR
    
    df_limpo = df_limpo[(df_limpo[col] >= limite_inf) & (df_limpo[col] <= limite_sup)]

print(f"   Extraido: {df_limpo.shape[0]:,} linhas x {df_limpo.shape[1]} colunas")
print(f"   Memoria: {df_limpo.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

   Extraido: 416,183 linhas x 36 colunas
   Memoria: 327.03 MB


# 5. Load (Carregamento dos dados para o Postgresql)

In [8]:
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()

In [12]:

cur.execute("TRUNCATE TABLE silver.user CASCADE;")
conn.commit()

In [ ]:
data = []
for index, row in df_limpo.iterrows():
     Time_value = None
     data.append((
           row['user_id'] if pd.notnull(row['user_id']) else None,
           row['age'] if pd.notnull(row['age']) else None,
           row['gender'] if pd.notnull(row['gender']) else None,
           row['country'] if pd.notnull(row['country']) else None,
           row['urban_rural'] if pd.notnull(row['urban_rural']) else None,
           row['income_level'] if pd.notnull(row['income_level']) else None,
           row['employment_status'] if pd.notnull(row['employment_status']) else None,
           row['relationship_status'] if pd.notnull(row['relationship_status']) else None,
           row['exercise_hours_per_week'] if pd.notnull(row['exercise_hours_per_week']) else None,
           row['sleep_hours_per_night'] if pd.notnull(row['sleep_hours_per_night']) else None,
           row['diet_quality'] if pd.notnull(row['diet_quality']) else None,
           row['smoking'] if pd.notnull(row['smoking']) else None,
           row['alcohol_frequency'] if pd.notnull(row['alcohol_frequency']) else None,
           row['perceived_stress_score'] if pd.notnull(row['perceived_stress_score']) else None,
           row['self_reported_happiness'] if pd.notnull(row['self_reported_happiness']) else None,
           row['body_mass_index'] if pd.notnull(row['body_mass_index']) else None,
           row['blood_pressure_systolic'] if pd.notnull(row['blood_pressure_systolic']) else None,
           row['blood_pressure_diastolic'] if pd.notnull(row['blood_pressure_diastolic']) else None,
           row['daily_steps_count'] if pd.notnull(row['daily_steps_count']) else None,
           row['weekly_work_hours'] if pd.notnull(row['weekly_work_hours']) else None,
           row['social_events_per_month'] if pd.notnull(row['social_events_per_month']) else None,
           row['books_read_per_year'] if pd.notnull(row['books_read_per_year']) else None,
           row['volunteer_hours_per_month'] if pd.notnull(row['volunteer_hours_per_month']) else None,
           row['daily_active_minutes_instagram'] if pd.notnull(row['daily_active_minutes_instagram']) else None,
           row['reels_watched_per_day'] if pd.notnull(row['reels_watched_per_day']) else None,
           row['stories_viewed_per_day'] if pd.notnull(row['stories_viewed_per_day']) else None,
           row['ads_viewed_per_day'] if pd.notnull(row['ads_viewed_per_day']) else None,
           row['ads_clicked_per_day'] if pd.notnull(row['ads_clicked_per_day']) else None,
           row['time_on_feed_per_day'] if pd.notnull(row['time_on_feed_per_day']) else None,
           row['time_on_explore_per_day'] if pd.notnull(row['time_on_explore_per_day']) else None,
           row['time_on_reels_per_day'] if pd.notnull(row['time_on_reels_per_day']) else None,
           row['followers_count'] if pd.notnull(row['followers_count']) else None,
           row['following_count'] if pd.notnull(row['following_count']) else None,
           row['content_type_preference'] if pd.notnull(row['content_type_preference']) else None,
           row['preferred_content_theme'] if pd.notnull(row['preferred_content_theme']) else None,
           row['user_engagement_score'] if pd.notnull(row['user_engagement_score']) else None
     ))

insert_sql = """
    INSERT INTO silver.user 
    (user_id, age, gender, country, urban_rural, income_level, 
        employment_status, relationship_status, exercise_hours_per_week, 
        sleep_hours_per_night, diet_quality, smoking, alcohol_frequency, 
        perceived_stress_score, self_reported_happiness, body_mass_index, 
        blood_pressure_systolic, blood_pressure_diastolic, 
        daily_steps_count, weekly_work_hours, social_events_per_month, 
        books_read_per_year, volunteer_hours_per_month, 
        daily_active_minutes_instagram, reels_watched_per_day, 
        stories_viewed_per_day, ads_viewed_per_day, ads_clicked_per_day, 
        time_on_feed_per_day, time_on_explore_per_day, 
        time_on_reels_per_day, followers_count, following_count, 
        content_type_preference, preferred_content_theme, 
        user_engagement_score)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
        %s, %s, %s, %s, %s, %s)
"""


execute_batch(cur, insert_sql, data, page_size=1000)
conn.commit()

print(f"{len(data):,} registros inseridos na silver.user")

416,183 registros inseridos na silver.product
